In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.69641, saving model to best_m.h5
 - 3s - loss: 0.6990 - acc: 0.5203 - val_loss: 0.6964 - val_acc: 0.5387
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.6462 - acc: 0.5908 - val_loss: 0.7002 - val_acc: 0.4938
Epoch 3/50
Epoch 00003: val_loss improved from 0.69641 to 0.53120, saving model to best_m.h5
 - 2s - loss: 0.6189 - acc: 0.6234 - val_loss: 0.5312 - val_acc: 0.7756
Epoch 4/50
Epoch 00004: val_loss improved from 0.53120 to 0.41428, saving model to best_m.h5
 - 2s - loss: 0.5485 - acc: 0.7253 - val_loss: 0.4143 - val_acc: 0.8130
Epoch 5/50
Epoch 00005: val_loss improved from 0.41428 to 0.35424, saving model to best_m.h5
 - 2s - loss: 0.5071 - acc: 0.7512 - val_loss: 0.3542 - val_acc: 0.8603
Epoch 6/50
Epoch 00006: val_loss improved from 0.35424 to 0.30210, saving model to best_m.h5
 - 2s - loss: 0.4348 - acc: 0.7933 - val_loss: 0.3021 - val_acc: 0.8429
Epoch 7/50
Epoch 00007: val_loss improved from 0.30210 to 

Epoch 11/50
Epoch 00011: val_loss improved from 0.34746 to 0.33769, saving model to best_m.h5
 - 2s - loss: 0.3381 - acc: 0.8623 - val_loss: 0.3377 - val_acc: 0.8479
Epoch 12/50
Epoch 00012: val_loss improved from 0.33769 to 0.29435, saving model to best_m.h5
 - 2s - loss: 0.2900 - acc: 0.8750 - val_loss: 0.2943 - val_acc: 0.8703
Epoch 13/50
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2818 - acc: 0.8875 - val_loss: 0.3178 - val_acc: 0.8579
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.3173 - acc: 0.8589 - val_loss: 0.3294 - val_acc: 0.8728
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.2792 - acc: 0.8841 - val_loss: 0.3151 - val_acc: 0.8803
Epoch 16/50
Epoch 00016: val_loss improved from 0.29435 to 0.29397, saving model to best_m.h5
 - 2s - loss: 0.2672 - acc: 0.8817 - val_loss: 0.2940 - val_acc: 0.8703
Epoch 17/50
Epoch 00017: val_loss improved from 0.29397 to 0.27477, saving model to best_m.h5
 - 2s - loss: 0.2589 - acc: 0.8900 - val_l

Epoch 20/50
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.3570 - acc: 0.8442 - val_loss: 0.3027 - val_acc: 0.8653
Epoch 21/50
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.3489 - acc: 0.8591 - val_loss: 0.3179 - val_acc: 0.8803
Epoch 22/50
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.3447 - acc: 0.8481 - val_loss: 0.3180 - val_acc: 0.8703
Epoch 23/50
Epoch 00023: val_loss improved from 0.26768 to 0.26655, saving model to best_m.h5
 - 2s - loss: 0.3462 - acc: 0.8483 - val_loss: 0.2665 - val_acc: 0.8903
Epoch 24/50
Epoch 00024: val_loss improved from 0.26655 to 0.25946, saving model to best_m.h5
 - 2s - loss: 0.3345 - acc: 0.8533 - val_loss: 0.2595 - val_acc: 0.8853
Epoch 25/50
Epoch 00025: val_loss improved from 0.25946 to 0.25662, saving model to best_m.h5
 - 2s - loss: 0.3276 - acc: 0.8450 - val_loss: 0.2566 - val_acc: 0.8953
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2785 - acc: 0.8808 - val_loss: 0.2877 - val_acc: 0.8828
Epoch 27/50
Ep

Epoch 31/50
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.2066 - acc: 0.9181 - val_loss: 0.3138 - val_acc: 0.8903
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.2180 - acc: 0.9083 - val_loss: 0.2983 - val_acc: 0.8903
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2206 - acc: 0.9092 - val_loss: 0.3073 - val_acc: 0.8953
Epoch 34/50
Epoch 00034: val_loss improved from 0.29217 to 0.29014, saving model to best_m.h5
 - 2s - loss: 0.2079 - acc: 0.9175 - val_loss: 0.2901 - val_acc: 0.8878
Epoch 35/50
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.2048 - acc: 0.9122 - val_loss: 0.3042 - val_acc: 0.8953
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 2s - loss: 0.2250 - acc: 0.9081 - val_loss: 0.3107 - val_acc: 0.8753
Epoch 37/50
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.2269 - acc: 0.9073 - val_loss: 0.2950 - val_acc: 0.9027
Epoch 38/50
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.1870 - acc: 0.9167 - val_loss: 0.300

In [6]:
import pickle
with open('../features/cnn_3_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug2_sub.csv', index=False)

0.228852774132
         id  is_iceberg
0  5941774d    0.047126
1  4023181e    0.476509
2  b20200e4    0.038597
3  e7f018bb    0.994452
4  4371c8c3    0.022245
